In [1]:
import redis
import torch
import json
import time
import uuid
from gtts import gTTS
from TTS.api import TTS
import os
from pathlib import Path
from IPython.display import Audio, display

In [2]:
r = redis.Redis()

In [3]:
audio_dir = Path("../public/audio")
audio_dir.mkdir(parents=True, exist_ok=True)

print("Worker started. Listening for tasks on Redis stream 'tts_stream'...")

Worker started. Listening for tasks on Redis stream 'tts_stream'...


In [ ]:
last_id = '0'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
while True:
    try:
        entries = r.xread({'tts_stream':last_id}, block=0, count=1)
        for stream, messages in entries:
            for msg_id, message in messages:
                last_id = msg_id.decode()
                task_id = message[b'task_id'].decode()
                text = message[b'text'].decode()
                model = message[b'model'].decode()
                language = message[b'language'].decode()

                print(f"Received Task {task_id}: '{text}'")

                r.hset(f'tts_status:{task_id}', mapping={
                    'status': 'processing'
                })

                try: 
                    filename = f"{task_id}.mp3"
                    filepath = audio_dir / filename
                    tts = TTS(model_name='tts_models/en/ljspeech/glow-tts').to(device)
                    tts.tts_to_file(text=text, file_path=str(filepath))
                    r.hset(f'tts_status:{task_id}', mapping={
                        'status': 'done',
                        'file': f'/audio/{filename}'
                    })
                    print(f"Task {task_id} done. File: {filepath}")
                    display(Audio(str(filepath)))
                except Exception as e:
                    r.hset(f'tts_status:{task_id}', mapping={
                        'status': 'error',
                        'message': str(e)
                    })
                    print(f"Error processing task {task_id}: {e}")
    except KeyboardInterrupt:
        print("worker stopped")
        break

Received Task 7bed5b78-50a7-4efc-bcd3-006fab143a5e: 'Hello, welcome to our platform.'
 > Downloading model to /home/immaduddin84/.local/share/tts/tts_models--en--ljspeech--glow-tts


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 344M/344M [23:05<00:00, 400kiB/s]

 > Model's license - MPL
 > Check https://www.mozilla.org/en-US/MPL/2.0/ for more info.
 > Downloading model to /home/immaduddin84/.local/share/tts/vocoder_models--en--ljspeech--multiband-melgan



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 344M/344M [23:15<00:00, 246kiB/s]

%|▏                                                                                                            | 116k/82.8M [00:00<01:15, 1.10MiB/s]
%|▎                                                                                                            | 247k/82.8M [00:00<01:09, 1.18MiB/s]
%|▌                                                                                                            | 383k/82.8M [00:00<01:05, 1.26MiB/s]
%|▋                                                                                                            | 510k/82.8M [00:00<01:14, 1.11MiB/s]
%|▊                                                                                                             | 624k/82.8M [00:00<01:53, 723kiB/s]
%|▉                                                                                                  